In [1]:
import tensorflow as tf
import tensorflow.keras as keras
from sklearn.model_selection import train_test_split
import unicodedata
import re
import numpy
import os
import io

2023-03-19 10:52:20.850379: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-19 10:52:21.097185: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-19 10:52:21.097213: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-19 10:52:21.154894: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-03-19 10:52:22.493459: W tensorflow/stream_executor/platform/de

In [2]:
path_to_zip = keras.utils.get_file('spa-eng.zip', origin='https://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip',extract=True)

In [3]:
path_to_zip

'/home/ali/.keras/datasets/spa-eng.zip'

In [4]:
path_to_file= os.path.join(os.path.dirname(path_to_zip), 'spa-eng' , 'spa.txt')
path_to_file

'/home/ali/.keras/datasets/spa-eng/spa.txt'

In [5]:
def unicode_to_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')

In [6]:
def preprocess_senetence(w):
    w= unicode_to_ascii(w.lower().strip())
    w = re.sub(r"([?.~,])", r" \1 ",w)
    w = re.sub(r"[' ']+", r" ",w)    
    w = re.sub(r"[^a-zA-Z?.~,]+"," ",w)
    w = w.rstrip().strip()
    w = '<start> ' + w + ' <end>' 
    return w

In [7]:
en_sen = 'I have sOMe friends to help.'
preprocess_senetence(en_sen)

'<start> i have some friends to help . <end>'

In [8]:
spa_sen= 'Él cambió de empleo.'
preprocess_senetence(spa_sen)

'<start> el cambio de empleo . <end>'

In [9]:
def create_dataset(path, num_examples):
    lines = io.open(path, encoding='UTF-8').read().strip().split('\n')
    word_pairs = [[preprocess_senetence(w) for w in l.split('\t')] for l in lines[:num_examples]]
    return zip(*word_pairs)

In [10]:
en,sp = create_dataset(path_to_file, None)

In [12]:
len(en)

118964

In [13]:
def max_length(tensor):
    return max(len(t) for t in tensor)

In [14]:
def tokenize(lang):
    lang_tokenizer = keras.preprocessing.text.Tokenizer(filters='')
    lang_tokenizer.fit_on_texts(lang)
    tensor = lang_tokenizer.texts_to_sequences(lang)
    tensor = keras.preprocessing.sequence.pad_sequences(tensor, padding='post')
    return tensor, lang_tokenizer

In [15]:
def load_dataset(path, num_examples=None):
    targ_lang, inp_lang = create_dataset(path, num_examples)
    input_tensor, inp_lang_tokenizer = tokenize(inp_lang)
    target_tensor, target_lang_tokenizer = tokenize(targ_lang)
    return input_tensor, target_tensor, inp_lang_tokenizer, target_lang_tokenizer

In [16]:
input_tensor, target_tensor, input_lang_tokenizer, target_lang_tokenizer = load_dataset(path_to_file, 20000)

In [17]:
input_tensor

array([[  1,  97,   3, ...,   0,   0,   0],
       [  1, 179,   3, ...,   0,   0,   0],
       [  1, 466,   3, ...,   0,   0,   0],
       ...,
       [  1,   7,  10, ...,   0,   0,   0],
       [  1,   7,  10, ...,   0,   0,   0],
       [  1,   7,  10, ...,   0,   0,   0]], dtype=int32)

In [18]:
max_length_targ, max_length_inp = max_length(target_tensor), max_length(input_tensor)

In [19]:
X_train, X_test, y_train, y_test = train_test_split(input_tensor, target_tensor, test_size=0.2)

In [20]:
def convert(lang, tensor):
    for t in tensor:
        if t != 0:
            print(t, ' .... ', lang.index_word[t])

In [21]:
convert(input_lang_tokenizer, input_tensor[0])

1  ....  <start>
97  ....  ve
3  ....  .
2  ....  <end>


In [22]:
input_tensor[0]

array([ 1, 97,  3,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
      dtype=int32)

In [23]:
target_lang_tokenizer.word_index

{'<start>': 1,
 '<end>': 2,
 '.': 3,
 'i': 4,
 'you': 5,
 '?': 6,
 'tom': 7,
 'is': 8,
 'it': 9,
 's': 10,
 'a': 11,
 'he': 12,
 't': 13,
 'the': 14,
 'we': 15,
 'm': 16,
 'me': 17,
 're': 18,
 'that': 19,
 'this': 20,
 'to': 21,
 'do': 22,
 'are': 23,
 'can': 24,
 'my': 25,
 'they': 26,
 'was': 27,
 'she': 28,
 'don': 29,
 'have': 30,
 'your': 31,
 'go': 32,
 'what': 33,
 'in': 34,
 'not': 35,
 'll': 36,
 'like': 37,
 'here': 38,
 'on': 39,
 'him': 40,
 'let': 41,
 'be': 42,
 'did': 43,
 'know': 44,
 'come': 45,
 'up': 46,
 'am': 47,
 'want': 48,
 'how': 49,
 ',': 50,
 'mary': 51,
 'get': 52,
 'who': 53,
 'very': 54,
 'now': 55,
 'need': 56,
 'has': 57,
 'please': 58,
 'no': 59,
 'there': 60,
 'help': 61,
 'her': 62,
 'love': 63,
 'at': 64,
 'see': 65,
 'just': 66,
 'out': 67,
 'his': 68,
 've': 69,
 'got': 70,
 'for': 71,
 'where': 72,
 'look': 73,
 'stop': 74,
 'one': 75,
 'us': 76,
 'good': 77,
 'car': 78,
 'too': 79,
 'so': 80,
 'all': 81,
 'why': 82,
 'will': 83,
 'an': 84,
 'hom

In [24]:
BUFFER_SIZE = len(X_train)
BATCH_SIZE = 64
steps_per_epoch = len(X_train) // BATCH_SIZE
embedding_dim = 256
units = 1024
vocab_inp_size = len(input_lang_tokenizer.word_index) + 1
vocab_targ_size = len(target_lang_tokenizer.word_index) + 1

In [26]:
dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

In [27]:
dataset

<BatchDataset element_spec=(TensorSpec(shape=(64, 16), dtype=tf.int32, name=None), TensorSpec(shape=(64, 10), dtype=tf.int32, name=None))>

In [28]:
class Encoder(keras.Model):
    def __init__(self, vocab_size, embedding_dim, enc_units, batch_size):
        super(Encoder, self).__init__()
        self.batch_size = batch_size
        self.enc_units = enc_units
        self.embedding = keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = keras.layers.GRU(self.enc_units, return_sequences=True, return_state=True)
    
    def call(self, x, hidden):
        x = self.embedding(x)
        output, state = self.gru(x, initial_state=hidden)
        return output, state
    
    def initilize_hidden_state(self):
        return tf.zeros((self.batch_size, self.enc_units))

In [29]:
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)

In [31]:
simple_hidden = encoder.initilize_hidden_state()
simple_hidden

<tf.Tensor: shape=(64, 1024), dtype=float32, numpy=
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)>

In [32]:
example_input_batch, example_target_batch = next(iter(dataset))

In [37]:
example_input_batch.shape

TensorShape([64, 16])

In [34]:
simple_output, simple_states = encoder(example_input_batch, simple_hidden)

In [36]:
simple_states

<tf.Tensor: shape=(64, 1024), dtype=float32, numpy=
array([[ 0.00527102,  0.02042053, -0.01350146, ..., -0.01555298,
        -0.01595373,  0.00287545],
       [ 0.00521931,  0.02048229, -0.01340425, ..., -0.0155583 ,
        -0.01594446,  0.00294792],
       [ 0.00524327,  0.02047041, -0.0133949 , ..., -0.01555656,
        -0.01591296,  0.00291451],
       ...,
       [ 0.00525062,  0.02045799, -0.01340994, ..., -0.01558557,
        -0.01594906,  0.00292271],
       [ 0.00528169,  0.02048117, -0.01353679, ..., -0.01553212,
        -0.01595937,  0.00285916],
       [ 0.00522701,  0.02039239, -0.01343617, ..., -0.01554746,
        -0.0159264 ,  0.0029066 ]], dtype=float32)>

In [41]:
class Attention(keras.layers.Layer):
    def __init__(self,units):
        super(Attention,self).__init__()
        self.W1 = keras.layers.Dense(units)
        self.W2 = keras.layers.Dense(units)
        self.V = keras.layers.Dense(1)
        
    def call(self, query, values):
        hidden_with_time_axis = tf.expand_dims(query, 1)
        score = self.V(tf.nn.tanh(self.W1(values)+ self.W1(hidden_with_time_axis)))
        atteion_weights =  tf.nn.softmax(score, axis=1)
        context_vector = atteion_weights* values
        context_vector = tf.reduce_sum(context_vector, axis=1)
        return context_vector, atteion_weights

In [43]:
attention_layer = Attention(10)
attetion_result, attetion_weights= attention_layer(simple_hidden,simple_output)

In [46]:
class Decoder(keras.Model):
     def __init__(self, vocab_size, embedding_dim, dec_units, batch_size):
        super(Decoder, self).__init__()
        self.batch_size = batch_size
        self.dec_units = dec_units
        self.embedding = keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = keras.layers.GRU(self.dec_units, return_sequences=True, return_state=True)
        self.fc = keras.layers.Dense(vocab_size)
        self.attention = Attention(self.dec_units)
    
     def call(self, x, hidden, enc_output):
        context_vector, attention_weights = self.attention(hidden, enc_output)
        x = self.embedding(x)
        x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)
        output, state = self.gru(x)
        output = tf.reshape(output, (-1, output.shape[2]))
        x = self.fc(output)
        return x, state, attention_weights

In [47]:
decoder = Decoder(vocab_targ_size, embedding_dim, units, BATCH_SIZE)

In [48]:
decoder(tf.random.uniform((BATCH_SIZE, 1)), simple_hidden, simple_output)

2023-03-19 11:17:58.438360: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 15728640 exceeds 10% of free system memory.
2023-03-19 11:17:58.447533: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 15728640 exceeds 10% of free system memory.
2023-03-19 11:17:58.451580: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 15728640 exceeds 10% of free system memory.
2023-03-19 11:17:59.852715: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 15265792 exceeds 10% of free system memory.
2023-03-19 11:17:59.861700: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 15265792 exceeds 10% of free system memory.


(<tf.Tensor: shape=(64, 3727), dtype=float32, numpy=
 array([[ 0.00279114, -0.00290082, -0.00242028, ..., -0.00365588,
          0.00288932, -0.00219868],
        [ 0.0028158 , -0.00246986, -0.00310751, ..., -0.00307487,
          0.00308768, -0.00242172],
        [ 0.00257201, -0.00206989, -0.00294965, ..., -0.00353961,
          0.00273038, -0.00231955],
        ...,
        [ 0.00330514, -0.00235843, -0.00272788, ..., -0.00349141,
          0.00307827, -0.00200316],
        [ 0.00303818, -0.00230001, -0.00225667, ..., -0.002949  ,
          0.00297231, -0.00211028],
        [ 0.00310373, -0.00280089, -0.00255609, ..., -0.00370273,
          0.00292987, -0.00208947]], dtype=float32)>,
 <tf.Tensor: shape=(64, 1024), dtype=float32, numpy=
 array([[ 6.0007949e-03, -1.1441417e-03, -1.1206528e-03, ...,
          1.1158745e-02, -1.4924384e-03, -6.7708497e-05],
        [ 6.4497632e-03, -2.0565877e-03, -1.7537407e-03, ...,
          9.7610541e-03, -1.7558716e-03, -4.2760788e-04],
        [ 5

In [49]:
optimizer = keras.optimizers.Adam()
loss_object = keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')

In [50]:
def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)
    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask
    return tf.reduce_mean(loss_)

In [51]:
checkpoint_dir = 'chckpnts'
checkpoint = tf.train.Checkpoint(optimizer=optimizer, encoder=encoder, decoder=decoder)

In [52]:
def train_step(inp, targ, enc_hidden):
    loss = 0
    with tf.GradientTape() as tape:
        enc_output, enc_hidden = encoder(inp, enc_hidden)
        dec_hidden = enc_hidden
        dec_input = tf.expand_dims([target_lang_tokenizer.word_index['<start>']] * BATCH_SIZE, 1)
        for t in range(1, targ.shape[1]):
            predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)
            loss  += loss_function(targ[:, t], predictions)
            dec_input = tf.expand_dims(targ[:, t], 1)
    batch_loss = (loss / int(targ.shape[1]))
    variables = encoder.trainable_variables + decoder.trainable_variables
    gradients = tape.gradient(loss, variables)
    optimizer.apply_gradients(zip(gradients, variables))
    return batch_loss

In [53]:
EPOCH = 10
for epoch in range(EPOCH):
    enc_hidden = encoder.initilize_hidden_state()
    total_loss = 0
    print('Epoch: ', epoch)

    for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
        batch_loss = train_step(inp, targ, enc_hidden)
        total_loss += batch_loss
    print('Loss: ', batch_loss.numpy())

    checkpoint.save(file_prefix='test1')

Epoch:  0
Loss:  4.4719563
Epoch:  0
Loss:  4.6309342
Epoch:  0
Loss:  4.619193
Epoch:  0
Loss:  4.380741
Epoch:  0
Loss:  3.267543
Epoch:  0
Loss:  4.088853
Epoch:  0
Loss:  3.87263
Epoch:  0
Loss:  3.2190711
Epoch:  0
Loss:  2.7756672
Epoch:  0
Loss:  2.8251655
Epoch:  0
Loss:  3.0252512
Epoch:  0
Loss:  2.9426816
Epoch:  0
Loss:  2.9381988
Epoch:  0
Loss:  3.0760682
Epoch:  0
Loss:  2.9412284
Epoch:  0
Loss:  2.94725
Epoch:  0
Loss:  2.7715602
Epoch:  0
Loss:  2.8958168
Epoch:  0
Loss:  2.8348436
Epoch:  0
Loss:  2.8854947
Epoch:  0
Loss:  2.7457404
Epoch:  0
Loss:  2.6557536
Epoch:  0
Loss:  2.6540906
Epoch:  0
Loss:  2.636438
Epoch:  0
Loss:  2.6955473
Epoch:  0
Loss:  2.6205125
Epoch:  0
Loss:  2.5753226
Epoch:  0
Loss:  2.5666661
Epoch:  0
Loss:  2.6777906
Epoch:  0
Loss:  2.64419
Epoch:  0
Loss:  2.561893
Epoch:  0
Loss:  2.560509
Epoch:  0
Loss:  2.6736207
Epoch:  0
Loss:  2.572294
Epoch:  0
Loss:  2.4399962
Epoch:  0
Loss:  2.5787406
Epoch:  0
Loss:  2.6100695
Epoch:  0
Loss:

In [54]:
def evaluate(sentence):
    sentence = preprocess_senetence(sentence)
    inputs = [input_lang_tokenizer.word_index[i] for i in sentence.split(' ')]
    inputs = keras.preprocessing.sequence.pad_sequences([inputs], maxlen=max_length_inp, padding='post')
    inputs = tf.convert_to_tensor(inputs)
    result = ''
    hidden = [tf.zeros((1, units))]
    enc_out, enc_hidden = encoder(inputs, hidden)
    dec_hidden = enc_hidden
    dec_input = tf.expand_dims([target_lang_tokenizer.word_index['<start>']], 0)
    for t in range(max_length_targ):
        predictions, dec_hidden, attention_weights = decoder(dec_input, dec_hidden, enc_out)
        attention_weights = tf.reshape(attention_weights, (-1, ))
        predicted_id = tf.argmax(predictions[0]).numpy()
        result += target_lang_tokenizer.index_word[predicted_id] + ' '
        if target_lang_tokenizer.index_word[predicted_id] == '<end>':
            return result, sentence
        dec_input = tf.expand_dims([predicted_id], 0)
    return result, sentence

In [55]:
checkpoint.restore(tf.train.latest_checkpoint(''))

In [62]:
evaluate('Hola cómo estás?')

('how are you are ? <end> ', '<start> hola como estas ? <end>')